# Notebook #3: Investigation on Biases against Item Providers' Fairness

**Context**. Considering the impact of recommendations on item providers is one of the duties of multi-sided recommender systems. Item providers are key stakeholders in online platforms, and their earnings and plans are influenced by the exposure their items receive in recommended lists. Prior work showed that certain minority groups of providers, characterized by a common sensitive attribute (e.g., gender or race), are being disproportionately affected by indirect and unintentional discrimination. This notebook will consider the directors of movies in Movielens 1M as the item providers and investigates how unfairness on gender-based demoraphic groups of providers affects providers' group visibility and exposure with respect to their representation in the item catalog.

**Motivation**. In this notebook, we explore the the idea of a group-level proportionality between the contribution in the catalog and the relevance, the visibility, and the exposure, following a distributive norm based on equity. Operationalizing this notion during the user-item relevance optimization stage may be envisioned as a proactive way of addressing provider’s fairness along the recommendation pipeline, given that relevance scores are the input for the ranking stage. Being optimized for their ability to rank, the estimated relevance scores directly influence the chance of an item being ranked high (i.e., the higher the relevance is, the more likely the item appears at the top). If these relevances are biased against the minority group, the recommender system is unfairly giving minority items less chance of being ranked high. The disparate impact we address consists in items of a small minority group of providers systematically receiving unfairly low relevance, and by extension an exposure not proportional to their contribution in the catalog. Our goal in this notebook is thus to investigate whether, during the recommendation learning stage, taking actions for increasing the relevance of items from the minority group positively impacts on providers’ group fairness.

**Problem Statement**. Under this scenario, to reduce disparities in relevance and exposure, we propose a pre-processing strategy that up-samples interactions where the minority group is predominant (e.g., an item where the minority is represented with two providers is better than item with only one provider of that group; moreover, the lower is the representation of the majority in that item, the more we can help the minority, by favoring an upsampling of these latter items). In addition, an in-processing component aims to control that the relevance to the items of the minority group is proportional to its contribution in the catalog. 

**Hands-on Outline**. In this notebook, we will focus on analyzing and mitigating unfairness against item providers through our introductory Python toolbox, in the simplest possible way. Specifically, we will:

- **Step 1** Load the working environment from GDrive. 

- **Step 2** Load the Movielens 1M dataset.

- **Step 3** Identify the representation of providers' groups in the catalog. 

- **Step 4** Investigate the visibility and exposure of the minority group of providers given by some recommender models. 

- **Step 5** Define and run a pre-processing treatment against provider unfairness, by upsampling interactions involving the minority. 

- **Step 6** Discuss the impact of our pre-processing treatment on the visibility and exposure of providers. 

- **Step 7** Inspect the impact of the amount of upsampling on the recommended lists. 

- **Step 8** Inspect the impact of the amount of upsampling on other recommendation metrics.  

**While gender is by no means a binary construct, to the best of our knowledge no dataset with non-binary genders exists. What we are considering is a binary feature, as the current publicly available platforms offer.**

## Step 1: Load the working environment from GDrive. 

This step serves to mount GDrive storage within this Jupyter notebook. The command will request us to give access permissions to this notebook, so that we will be able to clone the project repository when we desire. Please follow the prompted instructions.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

We move to the folder with the notebook in our tutorial codebase. 

In [ ]:
%cd /content/drive/My Drive/wsdm2021/notebooks

### Import Python packages

In [ ]:
import sys 
import os

sys.path.append(os.path.join('..'))

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from helpers.train_test_splitter import *
from helpers.instances_upsampler import *
from models.pointwise import PointWise
from models.pairwise_reg import PairWise
from models.mostpop import MostPop
from models.random import Random
from helpers.utils import *

### Define the base path to the pre-computed artifacts

In [ ]:
data_path = '../data/'

## Step 2: Load the Movielens 1M dataset.

The first step will require us to load the Movielens 1M dataset. 

In [ ]:
dataset = 'ml1m'          
method = 'utime_pfair'
user_field = 'user_id'
item_field = 'item_id'
rating_field = 'rating'
time_field = 'timestamp'
type_field = 'type_id'

In [ ]:
data = pd.read_csv('../data/datasets/' + dataset + '.csv', encoding='utf8')

In [ ]:
data.sample(n=10, random_state=1)

## Step 3: Identify the representation of providers' groups in the catalog. 

Now, in order to associate to each interaction the extent to which it involves each group of providers, we need to append the gender information to the interactions included in the original dataset. We will leverage a csv that, for each item, gives the percentage of directors belonging to the two genders. Please note that more than one director can be associated to a movie. 

In [ ]:
group_item_association = pd.read_csv('../data/datasets/' + dataset + '-dir-group' + '.csv', encoding='utf8')

In [ ]:
group_item_association.head()

In [ ]:
len(group_item_association[group_item_association['group_1'] > 0]) / len(group_item_association)

In [ ]:
len(group_item_association[group_item_association['group_1'] == 0]) / len(group_item_association)

We can observe that the first gender group represents a minority provider group in this dataset, with a representation of 5% in the catalog. The other gender group is reprensented by the 83% in the catalog. By summing up the two percentages, we do not reach a 100% of representation. This is due to the fact that, for some movies, we were not able to retrieve the gender information of the respective directors. For the sake of easiness, we assume that items of providers whose gender is unknown are part of the minority group, together with group 1. 

In [ ]:
group_item_association['minority'] = group_item_association['group_1'].apply(lambda x: 1.0 if math.isnan(x) else x)
group_item_association['majority'] = group_item_association['group_2'].apply(lambda x: 0.0 if math.isnan(x) else x)

In [ ]:
del group_item_association['group_1']
del group_item_association['group_2']

In [ ]:
len(group_item_association[group_item_association['minority'] > 0]) / len(group_item_association)

In [ ]:
len(group_item_association[group_item_association['majority'] > 0]) / len(group_item_association)

In this case study, we thus work with a minority having a representation of 16% in the catalog of items (conversely, the majority has a representation of 84%. Therefore, our goal would be to ensure that the visibility and the exposure of the minority group in the recommendations is proportional to such a representation in the catalog, i.e., 16%. 

In [ ]:
original_minority_rep = len(group_item_association[group_item_association['minority'] > 0]) / len(group_item_association)

In [ ]:
original_minority_rep

For the sake of easiness, we join the original interactions in Movielens 1M with the gender information of the providers involved in each interaction. 

In [ ]:
data = pd.merge(data, group_item_association, on='item_id')

In [ ]:
data.sample(n=10, random_state=1)

## Step 4: Investigate the visibility and exposure of the minority group of providers. 


In the first notebook, we run an experimental pipeline for four recommender models, namely Random, MostPop, PairWise, and PointWise. The output of each step of the pipeline has been saved into the folder *data/outputs/*. The goal of this step is to load the evaluation metrics associated to each recommender model and investigate how they vary across models. 

In [ ]:
data.sample(n=10, random_state=1)

In case you have successfully completed **Notebook #1**, you should have four metric files in *data/outputs/metrics*. 

In [ ]:
! ls "../data/outputs/metrics"

Hence, we have four recommender models, whose identifiers are the names of the corresponding pickle files. 

In [ ]:
model_types = ['utime_pairwise', 'utime_pointwise', 'utime_random', 'utime_mostpop']

For each recommender model, we load the pre-computed metrics in order to show them. This helps us to focus on the assessment of the unfairness behavior of the models, without requiring to re-run their experimental pipeline from scratch. 

In [ ]:
def load_metrics(model_types):
    metrics = {}
    for model_type in model_types:
        metrics[model_type] = load_obj(os.path.join(data_path, 'outputs/metrics/'+dataset+'_'+model_type+'_metrics.pkl'))
    return metrics

In [ ]:
metrics = load_metrics(model_types)

The same cut-offs considered in the previous notebooks will be used.  

In [ ]:
cutoffs = np.array([5, 10, 20, 50, 100, 200]) 

First, we compare the recommender models on the extent to which they provide effective recommendations to users. 

In [ ]:
def plot_effectiveness(metrics, cutoffs):
    plt.rcParams.update({'font.size': 16.5})
    plt.figure(figsize=(30, 7.5))

    plt.subplot(131)
    plt.title(r'Precision')
    plt.xlabel('Cutoff Value')
    plt.ylabel('Precision')
    for model_type in model_types:
        plt.plot(cutoffs, [np.mean(metrics[model_type]['precision'][k,:]) for k in range(len(cutoffs))], label=model_type)
    plt.xticks(cutoffs)
    plt.legend()
    plt.grid(axis='y')

    plt.subplot(132)
    plt.title(r'Recall')
    plt.xlabel('Cutoff Value')
    plt.ylabel('Recall')
    for model_type in model_types:
        plt.plot(cutoffs, [np.mean(metrics[model_type]['recall'][k,:]) for k in range(len(cutoffs))], label=model_type)
    plt.xticks(cutoffs)
    plt.legend()
    plt.grid(axis='y')

    plt.subplot(133)
    plt.title(r'NDCG')
    plt.xlabel('Cutoff Value')
    plt.ylabel('NDCG')
    for model_type in model_types:
        plt.plot(cutoffs, [np.mean(metrics[model_type]['ndcg'][k,:]) for k in range(len(cutoffs))], label=model_type)
    plt.xticks(cutoffs)
    plt.legend()
    plt.grid(axis='y')

    plt.tight_layout()

In [ ]:
plot_effectiveness(metrics, cutoffs)

Then, we show the extent to which they are fair for the providers, based on the group-proportionality principles discussed above.  

In [ ]:
def plot_disparities(metrics, cutoffs):
    plt.rcParams.update({'font.size': 16.5})
    plt.figure(figsize=(30, 7.5))

    plt.subplot(121)
    plt.title(r'Disparate Visibility')
    plt.xlabel('Cutoff Value')
    plt.ylabel('Disparate Visibility')
    for model_type in model_types:
        plt.plot(cutoffs, [np.mean(metrics[model_type]['visibility'][k,:]) - original_minority_rep for k in range(len(cutoffs))], label=model_type)
    plt.xticks(cutoffs)
    plt.legend()
    plt.grid(axis='y')

    plt.subplot(122)
    plt.title(r'Disparate Exposure')
    plt.xlabel('Cutoff Value')
    plt.ylabel('Disparate Exposure')
    for model_type in model_types:
        plt.plot(cutoffs, [np.mean(metrics[model_type]['exposure'][k,:]) - original_minority_rep for k in range(len(cutoffs))], label=model_type)
    plt.xticks(cutoffs)
    plt.legend()
    plt.grid(axis='y')

    plt.tight_layout()

In particular:

- We consider **visibility** as the percentage of providers of a given group in the recommendations (regardless of their position in the ranking), and **disparate visibility** as the absolute difference between the reprentation of the group in the catalog and the visibility received by the same group in the recommendations. 

- We consider **exposure** as the percentage of ranked-based attention received by a given group in the recommendations (counting their position in the ranking), and **disparate exposure** as the absolute difference between the reprentation of the group in the catalog and the exposure received by the same group in the recommendations. 

In [ ]:
plot_disparities(metrics, cutoffs)

## Step 5: Define and run a pre-processing treatment against provider unfairness. 

This part will show how to improve fairness among provider groups by upsampling interactions involving the minority group of providers. This example is a didactic version of the work proposed by Boratto et al. (2020b). 

**Boratto, L., Fenu, G., & Marras, M.**. (2020). Interplay between Upsampling and Regularization for Provider Fairness in Recommender Systems. arXiv preprint arXiv:2006.04279. [ArXiv Link](https://arxiv.org/abs/2006.04279)

First, we split again train and test data and prepare the data needed to initialize a recommendation model. 

In [ ]:
smode = 'utime_pfair'
train_ratio = 0.80        
min_train_samples = 8
min_test_samples = 2
min_time = None
max_time = None
step_time = 1000

In [ ]:
if smode == 'uftime_pfair':
    traintest = fixed_timestamp(data, min_train_samples, min_test_samples, min_time, max_time, step_time, user_field, item_field, time_field, rating_field)
elif smode == 'utime_pfair':
    traintest = user_timestamp(data, train_ratio, min_train_samples+min_test_samples, user_field, item_field, time_field)
elif smode == 'urandom_pfair':
    traintest = user_random(data, train_ratio, min_train_samples+min_test_samples, user_field, item_field)

We perform again the training and tes set split. 

In [ ]:
train = traintest[traintest['set']=='train'].copy()
test = traintest[traintest['set']=='test'].copy()

We compute the representation of the groups in the training set. 

In [ ]:
len(train[train['minority'] > 0]) / len(train), len(train[train['minority'] == 0]) / len(train)

In [ ]:
train.sample(10)

We prepare some data structures for item metadata. 

In [ ]:
users = list(np.unique(traintest[user_field].values))
items = list(np.unique(traintest[item_field].values))

In [ ]:
items_metadata = traintest.drop_duplicates(subset=['item_id'], keep='first')
category_per_item = items_metadata[type_field].values

Then, we identify the set of items belonging to the minority group and we run the upsampling strategy. 

In [ ]:
items_w_min = np.unique(traintest[traintest['minority'] > 0]['item_id'].values)
items_w_maj = np.unique(traintest[traintest['minority'] == 0]['item_id'].values)

In [ ]:
items_map = traintest.drop_duplicates(subset='item_id', keep='first')
item_group = {i: (0.0 if v > 0 else 1.0) for i, v in zip(items_map['item_id'].values, items_map['minority'].values)}

Finally, we define our pipeline for running upsampling of minority interactions before training the model. 

We present three different options for interaction upsampling:

- **real**: we upsample existing interactions for the minority group. 
- **fake**: we simulate novel interactions between users and items from the minority group. 
- **fakebypop**: we simulate novel interactions between users and items from the minority group, sampling thelatter by popularity. 

In [ ]:
def run_with_upsampling(train, umode, utarget, model_type='pairwise', items_w_min=None, item_group=None, cutoffs=None, verbose=False):
    model_id = smode + '_' + umode + '_' + str(int(utarget*100)) + '_' + model_type
    
    print('\n*** Running experimental pipeline for model with id', model_id)
    
    if umode == 'real':
        train = real(train, 'minority', target=utarget)
    if umode == 'fake':
        train = fake(train, 'minority', items_w_min, target=utarget)
    if umode == 'fakebypop':
        train = fakeByPop(train, 'minority', items_w_min, target=utarget)
        
    train['rating'] = 1.0
    
    model_types = {'random': Random, 'mostpop': MostPop, 'pointwise': PointWise, 'pairwise': PairWise}
    model = model_types[model_type](users, items, train, test, category_per_item, item_field, user_field, rating_field)
    
    model.train(no_epochs=5)
    model.predict()
    
    predictions = model.get_predictions()
    save_obj(predictions, os.path.join(data_path, 'outputs/predictions/' + dataset + '_' + model_id + '_scores.pkl'))
    
    model.test(item_group=item_group, cutoffs=cutoffs)
    
    metrics = model.get_metrics()
    
    save_obj(metrics, os.path.join(data_path, 'outputs/metrics/' + dataset + '_' + model_id + '_metrics.pkl'))
    
    if verbose:
        model.show_metrics(index_k=int(np.where(cutoffs == 10)[0]))

We run the pipeline to move the representation of the minority interactions from 0.07 to 0.16 (its representation in the catalog). 

In [ ]:
run_with_upsampling(train, 'fake', 0.16, 'pairwise', items_w_min, item_group, cutoffs, verbose=True)

## Step 6: Discuss the impact of our pre-processing treatment. 

Now, it is time to show how the pre-processing approaches has impacted the recommended lists. 

In [ ]:
model_types = ['utime_pfair_fake_16_pairwise', 'utime_pairwise', 'utime_random', 'utime_mostpop']

In [ ]:
metrics = load_metrics(model_types)

In [ ]:
plot_effectiveness(metrics, cutoffs)

In [ ]:
plot_disparities(metrics, cutoffs)

## Step 7: Inspect the impact of the amount of upsampling on the recommended lists. 

In the last step, we set up the amount of upsampling to 00.16. However, it remains unanswered the question on to what extent the upsampling affects the disparities and which amount of upsampling should be selected for a given model. To answer to this question, we run the procedure for a set of upsampling values and compare the tradeoff they achieve.  

In [ ]:
upsampling_vars = [0.20, 0.25]

In [ ]:
for u in upsampling_vars:
    run_with_upsampling(train, 'fake', u, 'pairwise', items_w_min, item_group, cutoffs)

Now, we load the metrics computed for the various upsampling degress and we compare the effectiveness and fairness metrics. 

In [ ]:
model_types=['utime_pairwise','utime_pfair_fake_16_pairwise','utime_pfair_fake_20_pairwise','utime_pfair_fake_25_pairwise']

In [ ]:
metrics = load_metrics(model_types)

In [ ]:
plot_effectiveness(metrics, cutoffs)

In [ ]:
plot_disparities(metrics, cutoffs)

## Step 8: Inspect the impact of the amount of upsampling on other metrics. 

In a lot of cases, it is important also to consider what has been the impact on other metrics. Here, we consider catalog coverage. 

In [ ]:
kindex = int(np.where(cutoffs == 5)[0])

In [ ]:
items_g1 = np.unique(traintest[traintest['minority'] > 0]['item_id'])
items_g2 = np.unique(traintest[traintest['minority'] == 0]['item_id'])

coverage_results = []
for model_type in model_types:
    coverage = len([1 for m in metrics[model_type]['item_coverage'][kindex] if m > 0]) / len(items)
    coverage_g1o = len([1 for m in metrics[model_type]['item_coverage'][kindex][items_g1] if m > 0]) / len(items)
    coverage_g2o = len([1 for m in metrics[model_type]['item_coverage'][kindex][items_g2] if m > 0]) / len(items)
    coverage_g1r = len([1 for m in metrics[model_type]['item_coverage'][kindex][items_g1] if m > 0]) / len(items_g1)
    coverage_g2r = len([1 for m in metrics[model_type]['item_coverage'][kindex][items_g2] if m > 0]) / len(items_g2)
    coverage_results.append([model_type, coverage, coverage_g1o, coverage_g2o, coverage_g1r, coverage_g2r])
    
pd.DataFrame(coverage_results, columns =['Model Type', 'Cov', 'G1 Over Cov', 'G2 Over Cov', 'G1 Rel Cov', 'G2 Rel Cov']) 

## Summary

In this notebook, we analyzed and mitigated unfairness against item providers. Specifically, we have loaded the working environment from GDrive and the Movielens 1M dataset, identified the representation of providers' groups in the catalog, investigated the visibility and exposure of the minority group of providers given by some recommender models, defined and run a pre-processing treatment against provider unfairness, by upsampling interactions involving the minority, discussed the impact of our pre-processing treatment on the visibility and exposure of providers, and finally inspected the impact of the amount of upsampling on the recommended lists on recommendation metrics.

## Suggested Reading

If you are interested in a full showcase of how to implement the proposed in-processing mitigation procedure, you could read:


**Boratto, L., Fenu, G., & Marras, M.**. (2020). Interplay between Upsampling and Regularization for Provider Fairness in Recommender Systems. arXiv preprint arXiv:2006.04279. [ArXiv Link](https://arxiv.org/abs/2006.04279)